# Explicación del Código para Clasificación de Cáncer de Mama con Pipelines y Clases

Este código utiliza **Pipelines** de `scikit-learn` y la organización en **clases** para realizar un flujo de trabajo de clasificación de cáncer de mama con tres modelos: **Regresión Logística**, **Árbol de Decisión**, y **Bosque Aleatorio**.

## Objetivos del Código:
1. **Cargar y preprocesar los datos**: El conjunto de datos de cáncer de mama se carga y se divide en entrenamiento y prueba.
2. **Entrenar varios modelos**: Se entrenan tres modelos diferentes usando pipelines.
3. **Evaluación de modelos**: Evaluar el rendimiento de cada modelo entrenado.
4. **Predicción de nuevos datos**: Hacer predicciones con nuevos datos de pacientes usando los modelos entrenados.

### Clase `CancerModeling`

La clase `CancerModeling` es responsable de manejar todo el flujo de trabajo, desde la carga de los datos hasta la predicción de nuevos pacientes.

#### Métodos de la clase:

1. **`__init__`**:
   - Carga el conjunto de datos de cáncer de mama.
   - Divide los datos en conjunto de entrenamiento y prueba.
   - Define los modelos que se utilizarán: **Regresión Logística**, **Árbol de Decisión** y **Bosque Aleatorio**.

2. **`crear_pipeline`**:
   - Crea un pipeline que incluye dos pasos:
     - **Estandarización** de los datos con `StandardScaler`.
     - El modelo elegido (Regresión Logística, Árbol de Decisión, o Bosque Aleatorio).
   - Esto asegura que el flujo de trabajo de preprocesamiento y modelado se realice de manera consistente.

3. **`entrenar_modelos`**:
   - Entrena cada modelo utilizando el pipeline creado en el método anterior.
   - Guarda los modelos entrenados en un diccionario `modelos_entrenados`.

4. **`evaluar_modelo`**:
   - Evalúa un modelo específico utilizando el conjunto de prueba.
   - Muestra la **exactitud**, el **reporte de clasificación** y la **matriz de confusión** para cada modelo.

5. **`evaluar_todos_modelos`**:
   - Evalúa todos los modelos entrenados y muestra el rendimiento de cada uno.

6. **`predecir_nuevo_paciente`**:
   - Realiza predicciones sobre un nuevo paciente utilizando el modelo entrenado.
   - Estandariza los datos del paciente y realiza la predicción, mostrando la clase predicha y su nombre.

### Flujo de Trabajo

1. **Instancia de la clase `CancerModeling`**:
   - Se crea una instancia de la clase para gestionar el flujo de trabajo de clasificación.

2. **Entrenamiento de modelos**:
   - Se entrenan los modelos con el método `entrenar_modelos`.

3. **Evaluación de modelos**:
   - Se evalúan todos los modelos utilizando el método `evaluar_todos_modelos`, mostrando la exactitud y otros indicadores de desempeño.

4. **Predicción de nuevos datos**:
   - Se pueden hacer predicciones con nuevos datos de pacientes utilizando el método `predecir_nuevo_paciente`.


In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

class CancerModeling:
    def __init__(self):
        # Cargar el dataset de cáncer de mama
        self.cancer = datasets.load_breast_cancer()
        self.X = pd.DataFrame(self.cancer.data, columns=self.cancer.feature_names)
        self.y = pd.Series(self.cancer.target, name='target')
        
        # Dividir los datos en entrenamiento y prueba
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.X, self.y, test_size=0.2, random_state=42)
        
        # Definir el pipeline base para la estandarización y los modelos
        self.modelos = {
            'Logistic Regression': LogisticRegression(max_iter=10000),
            'Decision Tree': DecisionTreeClassifier(),
            'Random Forest': RandomForestClassifier(n_estimators=1000)
        }
        
    def crear_pipeline(self, modelo):
        # Crear el pipeline con estandarización y modelo
        return Pipeline([
            ('scaler', StandardScaler()),
            ('modelo', modelo)
        ])
    
    def entrenar_modelos(self):
        # Entrenar los modelos usando pipeline
        self.modelos_entrenados = {}
        for nombre, modelo in self.modelos.items():
            print(f"Entrenando el modelo: {nombre}")
            pipeline = self.crear_pipeline(modelo)
            pipeline.fit(self.X_train, self.y_train)
            self.modelos_entrenados[nombre] = pipeline
            
    def evaluar_modelo(self, pipeline, X_test, y_test):
        # Evaluación del modelo entrenado
        y_pred = pipeline.predict(X_test)
        print("Exactitud:", accuracy_score(y_test, y_pred))
        print("\nReporte de clasificación:\n", classification_report(y_test, y_pred))
        cm = confusion_matrix(y_test, y_pred)
        print("\nMatriz de Confusión:\n", cm)

    def evaluar_todos_modelos(self):
        # Evaluar todos los modelos entrenados
        for nombre, pipeline in self.modelos_entrenados.items():
            print(f"\nEvaluación del modelo: {nombre}")
            self.evaluar_modelo(pipeline, self.X_test, self.y_test)
    
    def predecir_nuevo_paciente(self, modelo_nombre, paciente):
        # Predecir nuevos datos
        modelo = self.modelos_entrenados[modelo_nombre]
        paciente_scaled = modelo.named_steps['scaler'].transform([paciente])
        prediccion = modelo.predict(paciente_scaled)
        print(f"\nLa clase predicha es: {prediccion[0]}")
        print(f"Nombre de la clase: {self.cancer.target_names[prediccion[0]]}")

# Crear instancia de la clase
cancer_model = CancerModeling()

# Entrenar los modelos
cancer_model.entrenar_modelos()

# Evaluar todos los modelos entrenados
cancer_model.evaluar_todos_modelos()

# Supongamos que tenemos nuevos datos de un paciente
nuevo_paciente = cancer_model.X.iloc[0].values

# Realizar predicción con el modelo de Regresión Logística (por ejemplo)
cancer_model.predecir_nuevo_paciente('Logistic Regression', nuevo_paciente)


Entrenando el modelo: Logistic Regression
Entrenando el modelo: Decision Tree
Entrenando el modelo: Random Forest

Evaluación del modelo: Logistic Regression
Exactitud: 0.9736842105263158

Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.98      0.95      0.96        43
           1       0.97      0.99      0.98        71

    accuracy                           0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114


Matriz de Confusión:
 [[41  2]
 [ 1 70]]

Evaluación del modelo: Decision Tree
Exactitud: 0.9385964912280702

Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.91      0.93      0.92        43
           1       0.96      0.94      0.95        71

    accuracy                           0.94       114
   macro avg       0.93      0.94      0.93       114
weighted avg       0.94      0.94      0.94    

/home/gonzales06082002/monitoria_python/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/gonzales06082002/monitoria_python/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
